In [1]:
import pandas as pd
import numpy as np
import re
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from itertools import product
from scipy import stats

%matplotlib inline
plt.style.use('ggplot')
#plt.style.use('seaborn-pastel')
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

sys.path.append("..")
from utils.util_functions import *

In [2]:
event_df = make_df("20180101", "20181231")

In [3]:
team = "ロッテ"
lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))

In [4]:
entropy(np.array(lst)[:, 2])

2.4173431236188314

In [5]:
from scipy import stats
stats.entropy(pd.Series(np.array(lst)[:, 2]).value_counts().values)

2.4173431236188323

In [6]:
#pd.Series(np.array(lst)[:, 2]).value_counts().values

In [18]:
def entropy1(X):
    probs = [np.mean(X == c) for c in set(X)]
    return np.sum(-p * np.log(p) for p in probs)

In [64]:
cond_entropy(pd.DataFrame(lst).values[:, 2], pd.DataFrame(lst).values[:, [1]])

1.2855973628525774

In [71]:
np.array(lst)[:, [1, 2]]

array([['0010', '0011'],
       ['0011', '1011'],
       ['1011', '2011'],
       ..., 
       ['2010', '3000'],
       ['1000', '2000'],
       ['2000', '3000']],
      dtype='<U4')

In [81]:
drv.entropy_joint(np.array(lst)[:, [1, 2]].T, base=np.e) - drv.entropy(np.array(lst)[:, 1], base=np.e)

1.285597362852597

In [82]:
drv.entropy_joint(np.array(lst)[:, [0, 1, 2]].T, base=np.e) - drv.entropy_joint(np.array(lst)[:, [0, 1]].T, base=np.e)

1.1515042229999524

In [70]:
entropy(np.array(lst)[:, [1, 2]])

3.882518525390535

In [55]:
cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)

1.2855973628525774

In [83]:
cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values)

1.1515042230000212

In [51]:
drv.entropy_joint(X=np.array(lst)[:, [2]], base=np.e)

0.0

In [ ]:
def make_score_df(team):
    res = []
    columns = ["相手", "得点", "安打", "失点", "被安打"]
    files = os.listdir(SCORE_DATA_DIR)
    for file in files:
        score_df = pd.read_csv(os.path.join(SCORE_DATA_DIR, file), encoding="cp932", index_col=0)
        if not name_dict[team] in score_df.index:
            continue
        opponent = score_df.index[0] if score_df.index[1] == name_dict[team] else score_df.index[1]
        team_score = score_df.loc[name_dict[team], "計"]
        team_hits = score_df.loc[name_dict[team], "安"]
        opponent_score = score_df.loc[opponent, "計"]
        opponent_hits = score_df.loc[opponent, "安"]
        res.append([opponent, team_score, team_hits, opponent_score, opponent_hits])
    df = pd.DataFrame(res, columns=columns)
    df["得点率"] = df["得点"] / df["安打"]
    df["失点率"] = df["失点"] / df["被安打"]
    return df

In [ ]:
make_score_df("日本ハム").head()

In [ ]:
column = "得点"

In [ ]:
diffs = []
stds = []
for team in pacific:
    lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))
    # H(St+1 | St, St-1)
    second_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values)
    # H(St+1 | St)
    first_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)
    std = make_score_df(team).describe()[column]["std"] #/ make_score_df(team).describe()[column]["mean"]
    diffs.append(second_cond_entropy - first_cond_entropy)
    stds.append(std)
    print(team, second_cond_entropy - first_cond_entropy, std)

In [ ]:
plt.scatter(diffs, stds)

In [ ]:
diffs = []
stds = []
for team in central:
    lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))
    # H(St+1 | St, St-1)
    second_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values)
    # H(St+1 | St)
    first_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)
    std = make_score_df(team).describe()[column]["std"] #/ make_score_df(team).describe()[column]["mean"]
    diffs.append(second_cond_entropy - first_cond_entropy)
    stds.append(std)
    print(team, second_cond_entropy - first_cond_entropy, std)

In [ ]:
plt.scatter(diffs, stds)

# 得点で見るか失点で見るか / 安打数を入れてもいいかも
# 標準偏差をそのまま使うか / 変動係数を使うか

In [ ]:
diffs = []
stds = []
for team in pacific+central:
    lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))
    # H(St+1 | St, St-1)
    second_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values)
    # H(St+1 | St)
    first_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)
    std = make_score_df(team).describe()[column]["std"] / make_score_df(team).describe()[column]["mean"]
    diffs.append(second_cond_entropy - first_cond_entropy)
    stds.append(std)
    print(team, second_cond_entropy - first_cond_entropy, std)

In [ ]:
plt.scatter(diffs[:6], stds[:6], color="red")
plt.scatter(diffs[6:], stds[6:], color="blue")

In [ ]:
import statsmodels.api as sm

nsample = len(diffs)
X = np.column_stack((np.repeat(1, nsample), diffs))

model = sm.OLS(stds, X)
results = model.fit()

a, b = results.params

plt.scatter(diffs, stds)
plt.plot(diffs, a+b*np.array(diffs))
plt.show()

In [ ]:
results.pvalues

# MLBでの使い方

In [ ]:
files = os.listdir("/Users/konoharuki/Desktop/GitHub/baseball/game_detail_data_mlb")[1:]

In [ ]:
res = []
for file in files:
    res.append(pd.read_csv("/Users/konoharuki/Desktop/GitHub/baseball/game_detail_data_mlb/"+file, encoding="cp932", dtype=str).values.tolist())

In [ ]:
inning_list = []
for elem in res:
    inning_list.append(np.array(elem)[:, 1].tolist() + ["GAMESET"])

In [ ]:
#make_flattened_list(make_inning_triple(inning_list))

In [ ]:
event_df = make_df(data_dir=DETAIL_DATA_DIR_MLB)

In [ ]:
event_df